# Scratchwork: experimenting with "words"

In [2]:
import finite_algebras as alg

In [3]:
import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

In [4]:
z3 = alg.generate_cyclic_group(3)
z3

Group(
'Z3',
'Autogenerated cyclic Group of order 3',
['e', 'a', 'a^2'],
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]
)

In [5]:
z3.about()


** Group **
Name: Z3
Instance ID: 140286944534432
Description: Autogenerated cyclic Group of order 3
Order: 3
Identity: e
Commutative? Yes
Cyclic?: Yes
  Generators: ['a', 'a^2']
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       a     a^2       3
      2     a^2       a       3
Cayley Table (showing indices):
[[0, 1, 2], [1, 2, 0], [2, 0, 1]]


In [6]:
z3_sqr = z3 * z3
z3_sqr.about()


** Group **
Name: Z3_x_Z3
Instance ID: 140286944446544
Description: Direct product of Z3 & Z3
Order: 9
Identity: e:e
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0     e:e     e:e       1
      1     e:a   e:a^2       3
      2   e:a^2     e:a       3
      3     a:e   a^2:e       3
      4     a:a a^2:a^2       3
      5   a:a^2   a^2:a       3
      6   a^2:e     a:e       3
      7   a^2:a   a:a^2       3
      8 a^2:a^2     a:a       3
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [1, 2, 0, 4, 5, 3, 7, 8, 6],
 [2, 0, 1, 5, 3, 4, 8, 6, 7],
 [3, 4, 5, 6, 7, 8, 0, 1, 2],
 [4, 5, 3, 7, 8, 6, 1, 2, 0],
 [5, 3, 4, 8, 6, 7, 2, 0, 1],
 [6, 7, 8, 0, 1, 2, 3, 4, 5],
 [7, 8, 6, 1, 2, 0, 4, 5, 3],
 [8, 6, 7, 2, 0, 1, 5, 3, 4]]


In [7]:
alg.about_subalgebras(z3_sqr)


Subalgebras of <Group:Z3_x_Z3, ID:140286944446544>
  There is 1 unique proper subalgebra, up to isomorphism, out of 4 total subalgebras.
  as shown by the partitions below:

4 Isomorphic Commutative Normal Groups of order 3 with identity 'e:e':
      Group: Z3_x_Z3_subalgebra_0: ['e:e', 'e:a', 'e:a^2']
      Group: Z3_x_Z3_subalgebra_1: ['e:e', 'a:e', 'a^2:e']
      Group: Z3_x_Z3_subalgebra_2: ['e:e', 'a:a', 'a^2:a^2']
      Group: Z3_x_Z3_subalgebra_3: ['e:e', 'a:a^2', 'a^2:a']



[[Group(
  'Z3_x_Z3_subalgebra_0',
  'Subalgebra of: Direct product of Z3 & Z3',
  ['e:e', 'e:a', 'e:a^2'],
  [[0, 1, 2], [1, 2, 0], [2, 0, 1]]
  ),
  Group(
  'Z3_x_Z3_subalgebra_1',
  'Subalgebra of: Direct product of Z3 & Z3',
  ['e:e', 'a:e', 'a^2:e'],
  [[0, 1, 2], [1, 2, 0], [2, 0, 1]]
  ),
  Group(
  'Z3_x_Z3_subalgebra_2',
  'Subalgebra of: Direct product of Z3 & Z3',
  ['e:e', 'a:a', 'a^2:a^2'],
  [[0, 1, 2], [1, 2, 0], [2, 0, 1]]
  ),
  Group(
  'Z3_x_Z3_subalgebra_3',
  'Subalgebra of: Direct product of Z3 & Z3',
  ['e:e', 'a:a^2', 'a^2:a'],
  [[0, 1, 2], [1, 2, 0], [2, 0, 1]]
  )]]

In [8]:
ex = alg.Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  16 example algebras are available.
  Use "Examples[INDEX]" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-groups.m

## Klein-4 Group

In [9]:
v4 = ex[7]
v4

Group(
'V4',
'Klein-4 group',
['e', 'h', 'v', 'r'],
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]
)

In [10]:
v4.about()


** Group **
Name: V4
Instance ID: 140286951114064
Description: Klein-4 group
Order: 4
Identity: e
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1       h       h       2
      2       v       v       2
      3       r       r       2
Cayley Table (showing indices):
[[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]]


### Finite Presentation

$V_4 = \langle h, v | h^2 = v^2 = (hv)^2 = e \rangle$

In [65]:
import itertools as it

Generators and Relators:

In [163]:
def cartesian_products(elems):
    '''EXAMPLE: cartesian_products(['h', 'v']) ==> ['h_h', 'h_v', 'v_h', 'v_v']'''
    return map(lambda x: '_'.join(x), it.product(elems, elems))

def remove_items_from_list(lst, items):
    '''Return the list that results from removing all items in items from lst.'''
    if items == []:
        return lst
    else:
        return remove_items_from_list([x for x in lst if x != items[0]], items[1:])

def reduce_word_by_rel(word, rel):
    rel_ = rel + '_'
    _rel = '_' + rel
    if word == rel:
        return ''
    elif rel_ in word:
        return word.replace(rel_, '')
    elif _rel in word:
        return word.replace(_rel, '')
    else:
        return word
    
def reduce_word(word, rels):
    if rels == []:
        return word
    elif rels[0] in word:
        return reduce_word_by_rel(word, rels[0])
    else:
        return reduce_word(word, rels[1:])
    
def reduce_words(words, rels):
    return list(set(map(lambda x: reduce_word(x, rels), words)))

In [164]:
gens = ['h', 'v']
rels = ['h_h', 'v_v', 'h_v_h_v']

In [165]:
cartprod = list(cartesian_products(gens))
cartprod

['h_h', 'h_v', 'v_h', 'v_v']

In [166]:
new_items = remove_items_from_list(cartprod, rels)
new_items

['h_v', 'v_h']

In [167]:
elems = gens + new
elems

['h', 'v', 'h_v', 'v_h']

In [168]:
foo = list(cartesian_products(elems))
foo

['h_h',
 'h_v',
 'h_h_v',
 'h_v_h',
 'v_h',
 'v_v',
 'v_h_v',
 'v_v_h',
 'h_v_h',
 'h_v_v',
 'h_v_h_v',
 'h_v_v_h',
 'v_h_h',
 'v_h_v',
 'v_h_h_v',
 'v_h_v_h']

In [169]:
foo = remove_items_from_list(foo, elems + rels)
foo

['h_h_v',
 'h_v_h',
 'v_h_v',
 'v_v_h',
 'h_v_h',
 'h_v_v',
 'h_v_v_h',
 'v_h_h',
 'v_h_v',
 'v_h_h_v',
 'v_h_v_h']

In [172]:
foo2 = reduce_words(foo, rels)
foo2

['v_h_v_h', 'v_v', 'h', 'h_v_h', 'v', 'v_h_v', 'h_h']

In [173]:
foo3 = remove_items_from_list(foo2, elems + rels)
foo3

['v_h_v_h', 'h_v_h', 'v_h_v']